# 디스코드 봇 만들기

* 봇의 주 기능은 게임 파티원 모집이다.
* 봇을 이용하여 원하는 시간에 같이 게임 할 사람들을 모을 수 있도록 하는 것이 목적이다.


In [2]:
# 기본적인 봇 설정 코드
import discord
from discord.ext import commands
intents = discord.Intents.default()
intents.message_content = True

Reserve_bot = commands.Bot(command_prefix='!', intents=intents)

@Reserve_bot.event
async def on_ready():
    print(f'Logged in as {Reserve_bot.user}')

# Reserve_bot.run('YOUR_BOT_TOKEN_HERE')


# 기본적인 코드 설명

* command.Bot : 모듈에서 제공하는 클래스로 디스코드 봇을 쉽게 만들고 명령어를 정의할 수 있도록 돕는다.

* command_prefix='!' : 사용자가 봇과 상호작용할 때 명령어 앞에 붙이는 접두사를 지정한다. (예시: 사용자가 !hello 라고 입력하면 봇은 hello 명령어를 인식하고 실행한다.)

* @Reserve_bot.event : 데코레이터는 파이썬에서 추상 기능을 구현할 때 사용하는 것으로, 여기서는 봇의 이벤트를 처리할 함수를 정의한다.

* on_ready 함수 : 봇이 준비되었을 때 호출된다. 봇이 로그인되면 콘솔에 봇의 이름을 출력한다.

* Reserve_bot.run('봇의 토큰') : 디스코드 봇을 실행하는 함수로, 디스코드 개발자 포털에서 발급받은 봇 토큰을 입력한다.

In [5]:
#대략적인 모집 기능
import discord
from discord.ext import commands

Reserve_bot = commands.Bot(command_prefix='!', intents=intents)
recruitment_posts = {}

@Reserve_bot.event
async def on_ready():
    print(f'Logged in as {Reserve_bot.user}')

@Reserve_bot.command()
async def recruit(ctx, game: str, time: str, total_slots: int):
    embed = discord.Embed(title=f"{game} 모집", description=f"시작 시간: {time}\n현재 인원: 0/{total_slots}", color=0x00ff00)
    message = await ctx.send(embed=embed)
    await message.add_reaction("✅")
    recruitment_posts[message.id] = {
        "game": game,
        "time": time,
        "total_slots": total_slots,
        "current_slots": 0,
        "message": message
    }

@Reserve_bot.event
async def on_reaction_add(reaction, user):
    if user == Reserve_bot.user:
        return
    if reaction.message.id in recruitment_posts and reaction.emoji == "✅":
        post = recruitment_posts[reaction.message.id]
        if post["current_slots"] < post["total_slots"]:
            post["current_slots"] += 1
            embed = discord.Embed(title=f"{post['game']} 모집", description=f"시작 시간: {post['time']}\n현재 인원: {post['current_slots']}/{post['total_slots']}", color=0x00ff00)
            await post["message"].edit(embed=embed)

#Reserve_bot.run('YOUR_BOT_TOKEN_HERE')



# 대략적인 모집기능 코드 설명

* recruitment_posts = {} : 딕셔너리를 추가하여 모집 메시지를 관리한다.

* @Reserve_bot.command() : 데코레이터를 사용하여 새로운 명령어 recruit를 정의한다.
사용자가 명령어를 입력하면, game, time, total_slots 인자를 받아 discord.Embed 객체를 생성하고, 메시지를 디스코드 채널에 전송한다.
메시지에 "✅" 이모지를 추가하여 사람들이 반응할 수 있게 한다.
recruitment_posts 딕셔너리에 메시지 ID를 키로 사용하여 모집 정보를 저장한다.

* on_reaction_add : 이벤트 핸들러로 사용자가 메시지에 반응을 추가할 때 호출된다.
봇 자신이 반응한 경우를 제외하고, 반응이 "✅" 이모지이고, 해당 메시지가 recruitment_posts에 있는 경우에만 처리한다. recruitment_posts에서 모집 정보를 가져와 현재 인원을 증가시키고, 임베드 메시지를 업데이트하여 수정된 인원을 반영한다.

